- Table of contents
- [Todo App](#todo-app)
  - [Create prompt](#create-prompt)

# Todo App

- It can do following things on the behalf of user:
  - Create Todo
  - Update Todo
  - Delete Todo
  - Get Todo

## Create Prompt

- Here I am going to give context how they behave on user input.
- I use [**`few_shot_prompting`**](../../05_few_shot_prompting/few_shot_prompting.md), that them to learn from examples.
- I will give them few examples of how to behave on user input.

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage

system_message = """
"""
examples = []

prompt = ChatPromptTemplate.from_messages(
    [
        SystemMessage(content=system_message),
        *examples,
        ("human", "{input}")
    ]
)

## Creating Output Structure

In [ ]:
from typing import Optional, Union
from pydantic import BaseModel, Field
from typing_extensions import Annotated


class S1(BaseModel):
    pass


class S2(BaseModel):
    pass


class C(BaseModel):
    pass

## Creating Tools

In [ ]:
from langchain_core.tools import tool

## Initialize Model

In [2]:
from langchain_openai import ChatOpenAI

## Main Execution Function

In [3]:
def main():
    pass